In [6]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import itertools

### Scrape One Player

In [7]:
def scrape_player(URL: str):
    print(URL)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="athlete_bio")

    if results != None:
        table = results.find("table", class_="biodata")
        player_bio = {}
        player_bio["id"] = URL[-1]
        if results.find("img", class_="photo"):
            player_bio["photo_link"] = results.find("img", class_="photo")["src"]
        for player in table.findAll("tr"):
            # print(player.find("th").text.replace(" ", "_"))
            # print(player.find("td").text.lstrip())
            player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                "td"
            ).text.lstrip()
            # print(i.find("td").find("a"))
        games = soup.find("table", class_="table").findAll("tr", class_="active")
        game_l = []
        for game in games:
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game.findAll("td")[:3]],
                )
            )
            game_l.append(dict_cols)
        # player_bio["games"] = game_l

        a = soup.find("table", class_="table").findAll("tr")
        a2 = a[1:]
        # print(a[1:])
        # print(len(a[1:]))
        list_a = [b.has_attr("class") for b in a[1:]]
        print(list_a)
        list_b = np.nonzero(list_a)[0]
        print(list(itertools.pairwise(list_b)))
        print(len(list(itertools.pairwise(list_b))) + 1)
        list_c = []
        for i in list(itertools.pairwise(list_b)):
            # print(a2[i[0]:i[1]])
            list_c.append(a2[i[0] : i[1]])
        # print(a2[list_b[-1]:])
        list_c.append(a2[list_b[-1] :])
        len(list_c)
        game_list = []
        for game in list_c:
            # print(game[0])
            # print([i.text.replace("\n", "") for i in game[0].findAll("td")[:3]])
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game[0].findAll("td")[:3]],
                )
            )
            # print(dict_cols)
            events = []
            for event in game[1:]:
                # print([j.text.replace("\n", "") for j in event.findAll("td")[1:-2]])
                dict_event = dict(
                    zip(
                        ["event", "team", "position", "medal"],
                        [j.text.replace("\n", "") for j in event.findAll("td")[1:-2]],
                    )
                )
                events.append(dict_event)
            # print(events)
            dict_cols["Events"] = events
            # print(dict_cols)
            game_list.append(dict_cols)
        # print(game_list)
        player_bio["games"] = game_list

    return player_bio

In [8]:
scrape_player("https://www.olympedia.org/athletes/1")

https://www.olympedia.org/athletes/1
[True, False, False, True, False, False, False]
[(0, 3)]
2


{'id': '1',
 'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/1.jpg',
 'Roles': 'Competed in Olympic Games',
 'Sex': 'Male',
 'Full_name': 'François Joseph Marie Antoine "Jean-François"•Blanchy',
 'Used_name': 'Jean-François•Blanchy',
 'Born': '12 December 1886 in Bordeaux, Gironde (FRA)',
 'Died': '2 October 1960 in Saint-Jean-de-Luz, Pyrénées-Atlantiques (FRA)',
 'NOC': 'France',
 'games': [{'game': '1912 Summer Olympics',
   'discipline': 'Tennis',
   'noc': 'FRA',
   'Events': [{'event': 'Singles, Men(Olympic)',
     'team': '',
     'position': '=17',
     'medal': ''},
    {'event': 'Doubles, Men(Olympic)',
     'team': 'Jean Montariol',
     'position': 'DNS',
     'medal': ''}]},
  {'game': '1920 Summer Olympics',
   'discipline': 'Tennis',
   'noc': 'FRA',
   'Events': [{'event': 'Singles, Men(Olympic)',
     'team': '',
     'position': '=32',
     'medal': ''},
    {'event': 'Doubles, Mixed(Olympic)',
     'team': 'Jeanne Vaussard',
     'position': '=8',
     'medal': '

In [9]:
scrape_player("https://www.olympedia.org/athletes/93860")

https://www.olympedia.org/athletes/93860
[True, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False]
[(0, 2), (2, 11), (11, 20), (20, 28)]
5


{'id': '0',
 'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/93860.jpg',
 'Roles': 'Competed in Olympic Games • Other',
 'Sex': 'Male',
 'Full_name': 'Michael Fred•Phelps, II',
 'Used_name': 'Michael•Phelps',
 'Nick/petnames': 'The Baltimore Bullet',
 'Born': '30 June 1985 in Baltimore, Maryland (USA)',
 'Measurements': '193 cm / 91 kg',
 'Affiliations': 'NBAC, Baltimore (USA) / Michigan Wolverines, Ann Arbor (USA)',
 'NOC': 'United States',
 'games': [{'game': '2000 Summer Olympics',
   'discipline': 'Swimming (Aquatics)',
   'noc': 'USA',
   'Events': [{'event': '200 metres Butterfly, Men(Olympic)',
     'team': '',
     'position': '5',
     'medal': ''}]},
  {'game': '2004 Summer Olympics',
   'discipline': 'Swimming (Aquatics)',
   'noc': 'USA',
   'Events': [{'event': '200 metres Freestyle, Men(Olympic)',
     'team': '',
     'position': '3',
     'medal': 'Bronze'},
    {'event': '4 × 100 metres Freestyle Relay, Men(Olympic)',
     'team': 'United States',
     'position':

### Scrape All Olympic Games Participants

In [10]:
players = []
for i in range(1, 101):
    URL = f"https://www.olympedia.org/athletes/{i}"
    print(URL)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="athlete_bio")

    if results != None:
        table = results.find("table", class_="biodata")
        player_bio = {}
        player_bio["id"] = i
        if results.find("img", class_="photo"):
            player_bio["photo_link"] = results.find("img", class_="photo")["src"]
        for player in table.findAll("tr"):
            # print(player.find("th").text.replace(" ", "_"))
            # print(player.find("td").text.lstrip())
            player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                "td"
            ).text.lstrip()
            # print(i.find("td").find("a"))
        games = soup.find("table", class_="table").findAll("tr", class_="active")
        game_l = []
        for game in games:
            dict_cols = dict(
                zip(
                    ["game", "discipline", "noc"],
                    [i.text.replace("\n", "") for i in game.findAll("td")[:3]],
                )
            )
            game_l.append(dict_cols)
        player_bio["games"] = game_l

        players.append(player_bio)

with open("players.json", "w") as f:
    json.dump(players, f)

https://www.olympedia.org/athletes/1
https://www.olympedia.org/athletes/2
https://www.olympedia.org/athletes/3
https://www.olympedia.org/athletes/4
https://www.olympedia.org/athletes/5
https://www.olympedia.org/athletes/6
https://www.olympedia.org/athletes/7
https://www.olympedia.org/athletes/8
https://www.olympedia.org/athletes/9
https://www.olympedia.org/athletes/10
https://www.olympedia.org/athletes/11
https://www.olympedia.org/athletes/12
https://www.olympedia.org/athletes/13
https://www.olympedia.org/athletes/14
https://www.olympedia.org/athletes/15
https://www.olympedia.org/athletes/16
https://www.olympedia.org/athletes/17
https://www.olympedia.org/athletes/18
https://www.olympedia.org/athletes/19


KeyboardInterrupt: 

Problem: There are about 150 000 players to scrape all this data we need to split it into threads but also send these threads to different computers

### Scrape only players who won medals

We need selenium to disable the limit of displayed players and to get all medal havers

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [12]:
def get_medal_havers():
    URL = "https://www.olympedia.org/statistics/medal/athlete"
    page2 = requests.get(URL)

    soup2 = BeautifulSoup(page2.content, "html.parser")
    table = soup2.find("table", class_="table")

    col_names = [i.text for i in table.find("thead").findAll("th")]
    # print(col_names)

    # create webdriver object
    driver = webdriver.Firefox()
    driver.get("https://www.olympedia.org/statistics/medal/athlete")

    x = driver.find_element(By.NAME, "limit")
    drop = Select(x)

    # select by visible text
    drop.select_by_visible_text("Show all")
    # wait for all the players to be displayed on the site
    time.sleep(20)

    players = driver.find_elements(By.CLASS_NAME, "top")
    out = [i.get_attribute("innerHTML") for i in players]

    driver.close()

    players_medal = []
    for line in out:
        soup = BeautifulSoup(line, "html.parser")
        players_medal.append([i.text.lstrip() for i in soup.findAll("td")])

    df = pd.DataFrame(players_medal, columns=col_names)
    df.to_csv("medal_havers.csv", index=False)

    return df

In [13]:
get_medal_havers()

,Athlete,Nat,Gold,Silver,Bronze,Total
0,Michael Phelps,USA,23,3,2,28
1,Larisa Latynina,URS UKR,9,5,4,18
2,Paavo Nurmi,FIN,9,3,0,12
3,Mark Spitz,USA,9,1,1,11
4,Carl Lewis,USA,9,1,0,10
...,...,...,...,...,...,...
1083,Vinzenz Geiger,GER,2,1,0,3
1084,Hanna Öberg,SWE,2,1,0,3
1085,Eileen Gu,CHN,2,1,0,3
1086,Daiki Hashimoto,JPN,2,1,0,3


## Try Threading to get all the players

In [13]:
import threading
import logging

In [14]:
def scrape_players(r: int):
    logging.info(f"Starting Thread in range {r} - {r + 100}")
    players = []
    for i in range(r, r + 100):
        URL = f"https://www.olympedia.org/athletes/{str(i)}"
        # print(URL)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find("div", class_="athlete_bio")

        if results != None:
            table = results.find("table", class_="biodata")
            player_bio = {}
            player_bio["id"] = i
            if results.find("img", class_="photo"):
                player_bio["photo_link"] = results.find("img", class_="photo")["src"]
            for player in table.findAll("tr"):
                player_bio[player.find("th").text.replace(" ", "_")] = player.find(
                    "td"
                ).text.lstrip()

            games = soup.find("table", class_="table").findAll("tr")
            a2 = games[1:]
            list_a = [b.has_attr("class") for b in a2]
            list_b = np.nonzero(list_a)[0]

            list_c = []
            for i in list(itertools.pairwise(list_b)):
                list_c.append(a2[i[0] : i[1]])
            list_c.append(a2[list_b[-1] :])

            game_list = []
            for game in list_c:
                dict_cols = dict(
                    zip(
                        ["game", "discipline", "noc"],
                        [i.text.replace("\n", "") for i in game[0].findAll("td")[:3]],
                    )
                )
                events = []
                for event in game[1:]:
                    dict_event = dict(
                        zip(
                            ["event", "team", "position", "medal"],
                            [
                                j.text.replace("\n", "")
                                for j in event.findAll("td")[1:-2]
                            ],
                        )
                    )
                    events.append(dict_event)
                dict_cols["Events"] = events
                game_list.append(dict_cols)
            player_bio["games"] = game_list

            print(player_bio)

            players.append(player_bio)
    with open(f"players/players_{r}.json", "w") as f:
        json.dump(players, f)
    logging.info(f"Thread finished {r} - {r + 100}")
    return players

Inside each batch there is 1000 players and there are 150 batches, which is a lot and must be optimized

In [15]:
def thread_func(batch):
    thread_list = []
    for i in batch:
        thread = threading.Thread(target=scrape_players, args=(i,))
        thread_list.append(thread)
        thread.start()
    for t in thread_list:
        t.join()

In [16]:
batches = []
for i in list(range(1, 150000, 1000)):
    batches.append([i] + [y * 100 + i for y in range(1, 10)])

In [17]:
print(batches[1:])

[[149001, 149101, 149201, 149301, 149401, 149501, 149601, 149701, 149801, 149901]]


In [18]:
# for batch in batches:
#     thread_func(batch)

{'id': 149001, 'photo_link': 'https://d2a3o6pzho379u.cloudfront.net/149001.jpg', 'Roles': 'Competed in Olympic Games', 'Sex': 'Female', 'Full_name': 'Alexandra Elisabeth•Edebo', 'Used_name': 'Alexandra•Edebo', 'Born': '3 March 1996 in Solna, Stockholm (SWE)', 'Affiliations': 'IFK Mora  ', 'NOC': 'Sweden', 'games': [{'game': '2022 Winter Olympics', 'discipline': 'Freestyle Skiing (Skiing)', 'noc': 'SWE', 'Events': [{'event': 'Ski Cross, Women(Olympic)', 'team': '', 'position': '13', 'medal': ''}]}]}
{'id': 149201, 'Roles': 'Competed in Olympic Games', 'Sex': 'Female', 'Full_name': 'Nadine•Hofstetter', 'Used_name': 'Nadine•Hofstetter', 'Born': '21 October 1994 in Romoos, Luzern (SUI)', 'Measurements': '164 cm / 68 kg', 'Affiliations': 'SC Reinach, Reinach (SUI)', 'NOC': 'Switzerland', 'games': [{'game': '2022 Winter Olympics', 'discipline': 'Ice Hockey (Ice Hockey)', 'noc': 'SUI', 'Events': [{'event': 'Ice Hockey, Women(Olympic)', 'team': 'Switzerland', 'position': '4', 'medal': ''}]}]}
